In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [10]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Select
from bokeh.io import push_notebook
from IPython.display import display

output_notebook()

df = pd.read_csv("final_pricing_output.csv")
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')

lot_ids = sorted(df['ID'].unique().tolist())
initial_lot = lot_ids[0]

filtered_df = df[df['ID'] == initial_lot].sort_values('Timestamp')

source = ColumnDataSource(data={
    'x': filtered_df['Timestamp'],
    'model1': filtered_df['Price_Model1'],
    'model2': filtered_df['Price_Model2']
})

p = figure(title=f"Pricing Trend for Lot {initial_lot}", 
           x_axis_type='datetime', 
           width=800, height=400,
           x_axis_label="Time", 
           y_axis_label="Price ($)")

p.line('x', 'model1', source=source, line_width=2, color='blue', legend_label='Model 1')
p.line('x', 'model2', source=source, line_width=2, color='green', legend_label='Model 2')

p.legend.location = 'top_left'
p.legend.click_policy = 'hide'

select = Select(title="Select Parking Lot ID:", value=str(initial_lot), options=[str(i) for i in lot_ids])

def update_plot(attr, old, new):
    selected_lot = int(new)
    new_df = df[df['ID'] == selected_lot].sort_values('Timestamp')
    source.data = {
        'x': new_df['Timestamp'],
        'model1': new_df['Price_Model1'],
        'model2': new_df['Price_Model2']
    }
    p.title.text = f"Pricing Trend for Lot {selected_lot}"
    push_notebook()

select.on_change('value', update_plot)

display(column(select, p))

Loading BokehJS ...

Column(id='p1077', ...)